In [4]:
##定义函数，进行分词和去停词

import jieba
from tqdm import *

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r',encoding='utf-8').readlines()]
    return stopwords

def content_preprocess(Train_raw):
    #content：需要分词，需要删除停词的内容list#
    
    #读取停词
    filepath = 'stop_words_zh.txt'
    stopwords = stopwordslist(filepath)
    
    #开始分词
    content_all = []
    for i in tqdm(range(len(Train_raw))):
        content = Train_raw['content'][i]
        out_str = ''
        content_cut = jieba.cut(content, cut_all = True)
        for word in content_cut:
            if word not in stopwords:
                if word != '' and '\n' not in word:
                    out_str = out_str + ' ' + word
        out_str = out_str + '\n' 
        content_all.append(out_str)
    
    return content_all


In [5]:
##分别读取训练数据，验证数据和测试数据集
import pandas as pd

Train_data_raw = pd.read_csv('dataset\sentiment_analysis_trainingset.csv')
Validation_data_raw = pd.read_csv('dataset\sentiment_analysis_validationset.csv')
Test_data_raw = pd.read_csv('dataset\sentiment_analysis_testa.csv')

In [6]:
Train_content = content_preprocess(Train_data_raw)
Validation_content = content_preprocess(Validation_data_raw)
Test_content = content_preprocess(Test_data_raw)

  0%|                                               | 0/105000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18092482\AppData\Local\Temp\jieba.cache
Loading model cost 0.762 seconds.
Prefix dict has been built succesfully.
100%|███████████████████████████████████| 15000/15000 [00:37<00:00, 403.71it/s]


In [7]:
##对训练数据进行拟合，生成词典并保存，同时将验证和测试数据都转化为序列

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

print('正在处理文本')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(Train_content)
Train_sequence = tokenizer.texts_to_sequences(Train_content)
Vlidation_sequence = tokenizer.texts_to_sequences(Validation_content)
Test_sequence = tokenizer.texts_to_sequences(Test_content)

#保存生成的词典
print('正在生成词典并保存')
dictionary = tokenizer.word_index
file = 'dictionary.txt'
f = open(file,'w',encoding='utf-8')
f.write(str(dictionary))
f.close()


正在处理文本
正在生成词典并保存


In [8]:
#处理标签。

label_name = Train_data_raw.keys()
label_name = label_name.tolist()
label_name.remove('id')
label_name.remove('content')

def label_process(label_ori):
    label = label_ori.as_matrix()
    label[label == -1] = 2
    label[label == -2] = 3
    label_onehot = tf.keras.utils.to_categorical(label)
    return label_onehot
    
Train_labels = {}
Validation_labels = {}

#预处理训练集标签
for i in range(len(label_name)):
    tmp = label_process(Train_data_raw[label_name[i]])
    Train_labels[label_name[i]] = tmp
    
#预处理验证集标签
for i in range(len(label_name)):
    tmp = label_process(Validation_data_raw[label_name[i]])
    Validation_labels[label_name[i]] = tmp
    
print(Train_labels)

d:\python\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


{'location_traffic_convenience': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32), 'location_distance_from_business_district': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32), 'location_easy_to_find': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32), 'service_wait_time': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32), 'service_waiters_attitude': array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1

In [9]:
#使文本序列等长
import numpy as np
Train_pad = tf.keras.preprocessing.sequence.pad_sequences(Train_sequence, maxlen=100,value=0.0, padding = 'post')
Validation_pad = tf.keras.preprocessing.sequence.pad_sequences(Vlidation_sequence, maxlen=100,value=0.0, padding = 'post')
Test_pad = tf.keras.preprocessing.sequence.pad_sequences(Test_sequence, maxlen=100,value=0.0, padding = 'post')

Train_pad = np.array(Train_pad)
Validation_pad = np.array(Validation_pad)
Test_pad = np.array(Test_pad)

Train_pad[Train_pad>len(dictionary)-1] = 0
Validation_pad[Validation_pad>len(dictionary)-1] = 0
Test_pad[Test_pad>len(dictionary)-1] = 0

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Activation, Input, Flatten, BatchNormalization, Conv1D, MaxPooling1D, Concatenate, LSTM
def Base_model(input_node):
    c = Input(shape = (input_node,))
    a = Embedding(len(dictionary),256)(c)
#     conv_1 = Conv1D(filters = 6, kernel_size = 3, strides = 1, padding = 'same',activation = 'relu')(a)
#     conv_2 = Conv1D(filters = 6, kernel_size = 4, strides = 1, padding = 'same',activation = 'relu')(a)
#     conv_3 = Conv1D(filters = 6, kernel_size = 5, strides = 1, padding = 'same',activation = 'relu')(a)
    
#     pool_1 = MaxPooling1D(pool_size = 2, strides=1, padding = 'valid')(conv_1)
#     pool_2 = MaxPooling1D(pool_size = 2, strides=1, padding = 'valid')(conv_2)
#     pool_3 = MaxPooling1D(pool_size = 2, strides=1, padding = 'valid')(conv_3)
    
#     tmp_1 = Flatten()(pool_1)
#     tmp_2 = Flatten()(pool_2)
#     tmp_3 = Flatten()(pool_3)
    
#     tmp = Concatenate(axis = -1)([tmp_1, tmp_2, tmp_3])
#     feature = Dense(256, activation='relu')(tmp_1)
#     feature_bm = BatchNormalization()(feature)
    
    feature_1 = LSTM(256)(a)
    feature_2 = LSTM(256)(a)
    feature_3 = LSTM(256)(a)
    feature_4 = LSTM(256)(a)
    feature_5 = LSTM(256)(a)
    feature_6 = LSTM(256)(a)
    feature_7 = LSTM(256)(a)
    feature_8 = LSTM(256)(a)
    feature_9 = LSTM(256)(a)
    feature_10 = LSTM(256)(a)
    feature_11 = LSTM(256)(a)
    feature_12 = LSTM(256)(a)
    feature_13 = LSTM(256)(a)
    feature_14 = LSTM(256)(a)
    feature_15 = LSTM(256)(a)
    feature_16 = LSTM(256)(a)
    feature_17 = LSTM(256)(a)
    feature_18 = LSTM(256)(a)
    feature_19 = LSTM(256)(a)
    feature_20 = LSTM(256)(a)
    
    
    feature_11 = Dense(128, activation='softmax')(feature_1)
    feature_21 = Dense(128, activation='softmax')(feature_2)
    feature_31 = Dense(128, activation='softmax')(feature_3)
    feature_41 = Dense(128, activation='softmax')(feature_4)
    feature_51 = Dense(128, activation='softmax')(feature_5)
    feature_61 = Dense(128, activation='softmax')(feature_6)
    feature_71 = Dense(128, activation='softmax')(feature_7)
    feature_81 = Dense(128, activation='softmax')(feature_8)
    feature_91 = Dense(128, activation='softmax')(feature_9)
    feature_101 = Dense(128, activation='softmax')(feature_10)
    feature_111 = Dense(128, activation='softmax')(feature_11)
    feature_121 = Dense(128, activation='softmax')(feature_12)
    feature_131 = Dense(128, activation='softmax')(feature_13)
    feature_141 = Dense(128, activation='softmax')(feature_14)
    feature_151 = Dense(128, activation='softmax')(feature_15)
    feature_161 = Dense(128, activation='softmax')(feature_16)
    feature_171 = Dense(128, activation='softmax')(feature_17)
    feature_181 = Dense(128, activation='softmax')(feature_18)
    feature_191 = Dense(128, activation='softmax')(feature_19)
    feature_201 = Dense(128, activation='softmax')(feature_20)
    

    output_1 = Dense(4, activation='softmax')(feature_11)
    output_2 = Dense(4, activation='softmax')(feature_21)
    output_3 = Dense(4, activation='softmax')(feature_31)
    output_4 = Dense(4, activation='softmax')(feature_41)
    output_5 = Dense(4, activation='softmax')(feature_51)
    output_6 = Dense(4, activation='softmax')(feature_61)
    output_7 = Dense(4, activation='softmax')(feature_71)
    output_8 = Dense(4, activation='softmax')(feature_81)
    output_9 = Dense(4, activation='softmax')(feature_91)
    output_10 = Dense(4, activation='softmax')(feature_101)
    output_11 = Dense(4, activation='softmax')(feature_111)
    output_12 = Dense(4, activation='softmax')(feature_121)
    output_13 = Dense(4, activation='softmax')(feature_131)
    output_14 = Dense(4, activation='softmax')(feature_141)
    output_15 = Dense(4, activation='softmax')(feature_151)
    output_16 = Dense(4, activation='softmax')(feature_161)
    output_17 = Dense(4, activation='softmax')(feature_171)
    output_18 = Dense(4, activation='softmax')(feature_181)
    output_19 = Dense(4, activation='softmax')(feature_191)
    output_20 = Dense(4, activation='softmax')(feature_201)

    output_final = [output_1,output_2,output_3,output_4,output_5,output_6,
                   output_7,output_8,output_9,output_10,output_11,output_12,
                   output_13,output_14,output_15,output_16,output_17,output_18,
                   output_19,output_20]
    
    model = Model(inputs=c, outputs=output_final, name='base')
    print(model.summary())
    return model
#     model.add(Embedding(len(dictionary), 128, input_length=100))

In [15]:
from tensorflow.keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:


import numpy
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score


class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (numpy.asarray(self.model.predict(
            self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        return
metrics = Metrics()


In [17]:
model = Base_model(100)
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['categorical_accuracy'])
Train_label = []
Validation_label = []
for name in label_name:
    Train_label.append(Train_labels[name])
    Validation_label.append(Validation_labels[name])


    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 256)     26466816    input_2[0][0]                    
__________________________________________________________________________________________________
lstm_20 (LSTM)                  (None, 256)          525312      embedding_1[0][0]                
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 128)          32896       lstm_20[0][0]                    
__________________________________________________________________________________________________
lstm_21 (L

__________________________________________________________________________________________________
dense_41 (Dense)                (None, 128)          32896       lstm_39[0][0]                    
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 4)            516         dense_22[0][0]                   
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 4)            516         dense_23[0][0]                   
__________________________________________________________________________________________________
dense_44 (Dense)                (None, 4)            516         dense_24[0][0]                   
__________________________________________________________________________________________________
dense_45 (Dense)                (None, 4)            516         dense_25[0][0]                   
__________

In [ ]:
from sklearn import metrics
for i in range(5):
    name = 'model_v' + str(i)
    model.fit(Train_pad, Train_label, epochs=1, batch_size = 64)
    model.save(name)

    val_test = model.predict(Validation_pad)
    val_label = np.zeros((15000))

    y_pred = []
    y_true = []


    for i in range(len(label_name)):
        y_pred.append(np.argmax(val_test[i], axis = 1))
        y_true.append(np.argmax(Validation_labels[label_name[i]], axis=1))
    # val_label = np.copy(Validation_labels)

    from sklearn import metrics

    f1 = []
    for j in range(len(y_pred)):
        f1.append(metrics.f1_score(y_true[j], y_pred[j], average = 'micro'))

        
    print('================================================')
    print(np.average(f1))
    print('================================================')



d:\python\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


   448/105000 [..............................] - ETA: 20:02:07 - loss: 11.2517 - dense_42_loss: 0.5648 - dense_43_loss: 0.5616 - dense_44_loss: 0.5581 - dense_45_loss: 0.5639 - dense_46_loss: 0.5630 - dense_47_loss: 0.5662 - dense_48_loss: 0.5622 - dense_49_loss: 0.5602 - dense_50_loss: 0.5628 - dense_51_loss: 0.5597 - dense_52_loss: 0.5642 - dense_53_loss: 0.5626 - dense_54_loss: 0.5625 - dense_55_loss: 0.5629 - dense_56_loss: 0.5644 - dense_57_loss: 0.5626 - dense_58_loss: 0.5614 - dense_59_loss: 0.5627 - dense_60_loss: 0.5636 - dense_61_loss: 0.5624 - dense_42_categorical_accuracy: 0.0000e+00 - dense_43_categorical_accuracy: 0.4219 - dense_44_categorical_accuracy: 0.8281 - dense_45_categorical_accuracy: 0.0000e+00 - dense_46_categorical_accuracy: 0.1562 - dense_47_categorical_accuracy: 0.0156 - dense_48_categorical_accuracy: 0.1094 - dense_49_categorical_accuracy: 0.3594 - dense_50_categorical_accuracy: 0.0312 - dense_51_categorical_accuracy: 0.5781 - dense_52_categorical_accuracy: 

   896/105000 [..............................] - ETA: 15:28:23 - loss: 11.2281 - dense_42_loss: 0.5629 - dense_43_loss: 0.5605 - dense_44_loss: 0.5571 - dense_45_loss: 0.5615 - dense_46_loss: 0.5625 - dense_47_loss: 0.5639 - dense_48_loss: 0.5605 - dense_49_loss: 0.5595 - dense_50_loss: 0.5610 - dense_51_loss: 0.5588 - dense_52_loss: 0.5635 - dense_53_loss: 0.5613 - dense_54_loss: 0.5618 - dense_55_loss: 0.5618 - dense_56_loss: 0.5636 - dense_57_loss: 0.5620 - dense_58_loss: 0.5601 - dense_59_loss: 0.5611 - dense_60_loss: 0.5633 - dense_61_loss: 0.5613 - dense_42_categorical_accuracy: 0.0508 - dense_43_categorical_accuracy: 0.7188 - dense_44_categorical_accuracy: 0.7676 - dense_45_categorical_accuracy: 0.1074 - dense_46_categorical_accuracy: 0.3066 - dense_47_categorical_accuracy: 0.1074 - dense_48_categorical_accuracy: 0.2988 - dense_49_categorical_accuracy: 0.5000 - dense_50_categorical_accuracy: 0.3516 - dense_51_categorical_accuracy: 0.5898 - dense_52_categorical_accuracy: 0.0801 -

  1344/105000 [..............................] - ETA: 16:41:25 - loss: 11.2041 - dense_42_loss: 0.5613 - dense_43_loss: 0.5585 - dense_44_loss: 0.5558 - dense_45_loss: 0.5594 - dense_46_loss: 0.5619 - dense_47_loss: 0.5616 - dense_48_loss: 0.5586 - dense_49_loss: 0.5592 - dense_50_loss: 0.5593 - dense_51_loss: 0.5577 - dense_52_loss: 0.5628 - dense_53_loss: 0.5602 - dense_54_loss: 0.5610 - dense_55_loss: 0.5610 - dense_56_loss: 0.5631 - dense_57_loss: 0.5614 - dense_58_loss: 0.5590 - dense_59_loss: 0.5594 - dense_60_loss: 0.5624 - dense_61_loss: 0.5604 - dense_42_categorical_accuracy: 0.3906 - dense_43_categorical_accuracy: 0.7677 - dense_44_categorical_accuracy: 0.7583 - dense_45_categorical_accuracy: 0.4635 - dense_46_categorical_accuracy: 0.3563 - dense_47_categorical_accuracy: 0.4865 - dense_48_categorical_accuracy: 0.5490 - dense_49_categorical_accuracy: 0.5062 - dense_50_categorical_accuracy: 0.5490 - dense_51_categorical_accuracy: 0.6125 - dense_52_categorical_accuracy: 0.1552 -

...

  3584/105000 [>.............................] - ETA: 21:46:13 - loss: 11.0907 - dense_42_loss: 0.5537 - dense_43_loss: 0.5507 - dense_44_loss: 0.5492 - dense_45_loss: 0.5493 - dense_46_loss: 0.5601 - dense_47_loss: 0.5493 - dense_48_loss: 0.5494 - dense_49_loss: 0.5577 - dense_50_loss: 0.5525 - dense_51_loss: 0.5541 - dense_52_loss: 0.5595 - dense_53_loss: 0.5543 - dense_54_loss: 0.5572 - dense_55_loss: 0.5569 - dense_56_loss: 0.5606 - dense_57_loss: 0.5579 - dense_58_loss: 0.5531 - dense_59_loss: 0.5517 - dense_60_loss: 0.5575 - dense_61_loss: 0.5559 - dense_42_categorical_accuracy: 0.6587 - dense_43_categorical_accuracy: 0.7797 - dense_44_categorical_accuracy: 0.7547 - dense_45_categorical_accuracy: 0.7500 - dense_46_categorical_accuracy: 0.3944 - dense_47_categorical_accuracy: 0.8016 - dense_48_categorical_accuracy: 0.7569 - dense_49_categorical_accuracy: 0.4956 - dense_50_categorical_accuracy: 0.6903 - dense_51_categorical_accuracy: 0.6103 - dense_52_categorical_accuracy: 0.4184 -

  4032/105000 [>.............................] - ETA: 22:05:54 - loss: 11.0672 - dense_42_loss: 0.5521 - dense_43_loss: 0.5489 - dense_44_loss: 0.5477 - dense_45_loss: 0.5472 - dense_46_loss: 0.5597 - dense_47_loss: 0.5470 - dense_48_loss: 0.5476 - dense_49_loss: 0.5573 - dense_50_loss: 0.5511 - dense_51_loss: 0.5533 - dense_52_loss: 0.5589 - dense_53_loss: 0.5531 - dense_54_loss: 0.5564 - dense_55_loss: 0.5560 - dense_56_loss: 0.5601 - dense_57_loss: 0.5572 - dense_58_loss: 0.5518 - dense_59_loss: 0.5500 - dense_60_loss: 0.5567 - dense_61_loss: 0.5551 - dense_42_categorical_accuracy: 0.6768 - dense_43_categorical_accuracy: 0.7845 - dense_44_categorical_accuracy: 0.7582 - dense_45_categorical_accuracy: 0.7675 - dense_46_categorical_accuracy: 0.3975 - dense_47_categorical_accuracy: 0.8169 - dense_48_categorical_accuracy: 0.7678 - dense_49_categorical_accuracy: 0.4962 - dense_50_categorical_accuracy: 0.6996 - dense_51_categorical_accuracy: 0.6140 - dense_52_categorical_accuracy: 0.4323 -

  4480/105000 [>.............................] - ETA: 22:28:58 - loss: 11.0451 - dense_42_loss: 0.5505 - dense_43_loss: 0.5476 - dense_44_loss: 0.5463 - dense_45_loss: 0.5452 - dense_46_loss: 0.5595 - dense_47_loss: 0.5444 - dense_48_loss: 0.5458 - dense_49_loss: 0.5570 - dense_50_loss: 0.5498 - dense_51_loss: 0.5528 - dense_52_loss: 0.5584 - dense_53_loss: 0.5521 - dense_54_loss: 0.5556 - dense_55_loss: 0.5552 - dense_56_loss: 0.5596 - dense_57_loss: 0.5566 - dense_58_loss: 0.5506 - dense_59_loss: 0.5484 - dense_60_loss: 0.5557 - dense_61_loss: 0.5541 - dense_42_categorical_accuracy: 0.6877 - dense_43_categorical_accuracy: 0.7808 - dense_44_categorical_accuracy: 0.7576 - dense_45_categorical_accuracy: 0.7793 - dense_46_categorical_accuracy: 0.3943 - dense_47_categorical_accuracy: 0.8320 - dense_48_categorical_accuracy: 0.7766 - dense_49_categorical_accuracy: 0.4961 - dense_50_categorical_accuracy: 0.7056 - dense_51_categorical_accuracy: 0.6079 - dense_52_categorical_accuracy: 0.4363 -

  4928/105000 [>.............................] - ETA: 22:49:30 - loss: 11.0230 - dense_42_loss: 0.5491 - dense_43_loss: 0.5461 - dense_44_loss: 0.5449 - dense_45_loss: 0.5432 - dense_46_loss: 0.5592 - dense_47_loss: 0.5421 - dense_48_loss: 0.5440 - dense_49_loss: 0.5568 - dense_50_loss: 0.5485 - dense_51_loss: 0.5520 - dense_52_loss: 0.5578 - dense_53_loss: 0.5510 - dense_54_loss: 0.5548 - dense_55_loss: 0.5541 - dense_56_loss: 0.5591 - dense_57_loss: 0.5559 - dense_58_loss: 0.5495 - dense_59_loss: 0.5470 - dense_60_loss: 0.5548 - dense_61_loss: 0.5533 - dense_42_categorical_accuracy: 0.6943 - dense_43_categorical_accuracy: 0.7810 - dense_44_categorical_accuracy: 0.7597 - dense_45_categorical_accuracy: 0.7892 - dense_46_categorical_accuracy: 0.3922 - dense_47_categorical_accuracy: 0.8415 - dense_48_categorical_accuracy: 0.7830 - dense_49_categorical_accuracy: 0.4932 - dense_50_categorical_accuracy: 0.7095 - dense_51_categorical_accuracy: 0.6107 - dense_52_categorical_accuracy: 0.4428 -

  5248/105000 [>.............................] - ETA: 23:10:14 - loss: 10.9997 - dense_42_loss: 0.5475 - dense_43_loss: 0.5443 - dense_44_loss: 0.5433 - dense_45_loss: 0.5411 - dense_46_loss: 0.5589 - dense_47_loss: 0.5397 - dense_48_loss: 0.5423 - dense_49_loss: 0.5566 - dense_50_loss: 0.5470 - dense_51_loss: 0.5512 - dense_52_loss: 0.5572 - dense_53_loss: 0.5498 - dense_54_loss: 0.5540 - dense_55_loss: 0.5531 - dense_56_loss: 0.5586 - dense_57_loss: 0.5552 - dense_58_loss: 0.5483 - dense_59_loss: 0.5453 - dense_60_loss: 0.5539 - dense_61_loss: 0.5524 - dense_42_categorical_accuracy: 0.7047 - dense_43_categorical_accuracy: 0.7847 - dense_44_categorical_accuracy: 0.7640 - dense_45_categorical_accuracy: 0.7983 - dense_46_categorical_accuracy: 0.3898 - dense_47_categorical_accuracy: 0.8500 - dense_48_categorical_accuracy: 0.7873 - dense_49_categorical_accuracy: 0.4900 - dense_50_categorical_accuracy: 0.7153 - dense_51_categorical_accuracy: 0.6130 - dense_52_categorical_accuracy: 0.4497 -

In [ ]:
ans = model.predict(Test_pad)

In [29]:
for j in range(len(label_name)):
    tmp = np.array(ans[j])
    tmp = np.argmax(tmp,axis=1)
    tmp[tmp == 3] = -2
    tmp[tmp == 2] = -1
    Test_data_raw[label_name[j]] = tmp

In [30]:
Test_data_raw.to_csv("submission_v3.csv",index = False)

In [31]:
Test_data_raw

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",1,-2,-2,-2,1,-2,-2,1,...,1,-2,-2,1,-2,1,-2,-2,1,1
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",1,-2,-2,-2,1,-2,-2,1,...,1,-2,-2,-2,-2,1,-2,-2,1,1
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",1,1,1,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,1,-2,-2,1,1
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",1,1,1,-2,1,-2,-2,-2,...,1,1,1,1,-2,1,-2,-2,1,1
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",1,-2,-2,-2,1,-2,-2,-2,...,1,1,1,1,-2,1,1,-2,1,1
5,5,"""尽管韩国烤肉店在无锡已经有很多家了，但因为味道好吃再加上在无锡很有人气，依旧挡不住新店的开...",-2,-2,-2,-2,1,-2,-2,-2,...,1,-2,-2,-2,-2,1,-2,-2,1,1
6,6,"""店铺在乙烯生活二区的西北角，旁边是国旅的旗舰店，门口就是红绿灯，挺好找的。因为这是两磅一的...",1,-2,1,-2,1,-2,-2,1,...,-2,-2,-2,1,1,1,-2,-2,1,1
7,7,"""朋友聚会来滴，这个地方真心不错哇，又能撸串，又能唱歌，还能看现场歌手演唱。最主要是可以上台...",1,-2,-2,-2,1,-2,-2,-2,...,1,1,1,1,-2,1,1,-2,1,1
8,8,"""超喜欢这家的面食，经常来吃。这家风格及产品都很像京城御面，有时会怀疑是不是同一家的。面条比...",-2,-2,-2,-2,-2,-2,-2,1,...,1,-2,-2,-2,1,1,-2,-2,1,1
9,9,"""广西阳朔，前一天晚上吃的特色啤酒鱼，就看到了隔壁有家螺蛳粉，第二天过来吃。个人比较喜欢粉，...",-2,-2,-2,-2,1,-2,-2,-2,...,1,-2,-2,1,1,1,-2,-2,1,1
